# Memory
Memory allows you to bring previous messages, instructions and responses into the context of a new prompt. This is particularly useful for chatbots and conversation chains.

We can start by initiating the LLM object with a built-in OpenAI chat model.

In [2]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    openai_api_key="KEY-ABC-DEF",
    model_name='gpt-3.5-turbo'
)

Some orchestration frameworks (like LangChain) have callback handlers to monitor and log LLM tasks. Here we can monitor how many tokens have been submitted.

In [3]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f"Spent a total of {cb.total_tokens} tokens")
    
    return result

We can then define the conversation chain with a memory argument. In this case we are using Summary Memory, which is useful for keeping token counts down on longer conversations. Summary memory makes use of an LLM to create the summary, so that should also be considered in the final token count.

In [4]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In the following prompts, watch how the conversation history element changes in the LLM response.

In [7]:
prompt = "Tell me something about weather in 2018."
conversation_sum(prompt)

{'input': 'Tell me something about weather in 2018.',
 'history': '',
 'response': 'In 2018, there were several significant weather events around the world. For example, there were devastating wildfires in California, record-breaking heatwaves in Europe, and powerful hurricanes in the Atlantic. Overall, 2018 was one of the hottest years on record globally, with many regions experiencing extreme weather patterns. Is there anything specific you would like to know about the weather in 2018?'}

In [9]:
prompt = "Were there any particularly long lasting storms in 2018?"
conversation_sum(prompt)

{'input': 'Were there any particularly long lasting storms in 2018?',
 'history': 'The human asks the AI to share information about the weather in 2018. The AI explains that there were significant weather events globally, such as wildfires in California, heatwaves in Europe, and hurricanes in the Atlantic. Overall, 2018 was one of the hottest years on record with extreme weather patterns.',
 'response': 'Yes, in 2018 there were several long-lasting storms that made headlines. For example, Hurricane Florence lingered over the Carolinas for days, causing catastrophic flooding. Additionally, Typhoon Mangkhut in the western Pacific was one of the strongest storms of the year and caused widespread damage in the Philippines and China. These storms were just a few examples of the extreme weather events that occurred in 2018.'}

In [11]:
prompt = "What were we just talking about?"
conversation_sum(prompt)

{'input': 'What were we just talking about?',
 'history': 'The human asks the AI to share information about the weather in 2018. The AI explains that there were significant weather events globally, such as wildfires in California, heatwaves in Europe, and hurricanes in the Atlantic, making it one of the hottest years on record with extreme weather patterns. In addition, the AI mentions that there were long-lasting storms in 2018, including Hurricane Florence and Typhoon Mangkhut, which caused catastrophic flooding and widespread damage in the Carolinas, Philippines, and China.',
 'response': 'We were discussing the weather in 2018, which was marked by significant events like wildfires in California, heatwaves in Europe, and hurricanes in the Atlantic. It was one of the hottest years on record with extreme weather patterns, including long-lasting storms like Hurricane Florence and Typhoon Mangkhut.'}

In [12]:
print(conversation_sum.memory.chat_memory.messages)

[HumanMessage(content='Tell me something about weather in 2018.'), AIMessage(content='In 2018, there were several significant weather events around the world. For example, there were devastating wildfires in California, record-breaking heatwaves in Europe, and powerful hurricanes in the Atlantic. Overall, 2018 was one of the hottest years on record globally, with many regions experiencing extreme weather patterns. Is there anything specific you would like to know about the weather in 2018?'), HumanMessage(content='Were there any particularly long lasting storms in 2018?'), AIMessage(content='Yes, in 2018 there were several long-lasting storms that made headlines. For example, Hurricane Florence lingered over the Carolinas for days, causing catastrophic flooding. Additionally, Typhoon Mangkhut in the western Pacific was one of the strongest storms of the year and caused widespread damage in the Philippines and China. These storms were just a few examples of the extreme weather events th

For applications with recurring users, or long lasting conversations, a database can be used to persist conversation histories. Here we will store each history entry in a seperate row, for simple access with SQL.

In [30]:
import iris     # We can use the same IRIS database running in the container
conn = iris.connect(hostname='localhost', 
                    port=51972, 
                    namespace='USER',
                    username='SuperUser', 
                    password='SYS')

# Create a table to store message prompts and responses.
cursor = conn.cursor()
cursor.execute("""
        CREATE TABLE RAG_Application.ConversationHistory (
            Username VARCHAR(100),
            ConversationTitle VARCHAR(250),
            HumanMessage VARCHAR(10000),
            AIMessage VARCHAR(10000)
        )
    """)
cursor.close()

Now we can store the conversation messages in a relational table, just using SQL:

In [24]:
username = input("Enter your username")
chat_history = conversation_sum.memory.chat_memory.messages

cursor = conn.cursor()

for i in range(0,len(chat_history),2):
    conversation_title = "This title"
    human_message = chat_history[i].content
    ai_message = chat_history[i+1].content
    cursor.execute(f"""INSERT INTO RAG_Application.ConversationHistory (Username, ConversationTitle, HumanMessage, AIMessage)
                   VALUES ('{username}', '{conversation_title}', '{human_message}', '{ai_message}')""")

cursor.close()

Retrieve the persisted conversation:

In [29]:
cursor = conn.cursor()
cursor.execute(f"""SELECT HumanMessage, AIMessage FROM RAG_Application.ConversationHistory 
               WHERE Username = '{username}' AND ConversationTitle = '{conversation_title}'""")

for row in cursor.fetchall():
    print(row)

cursor.close()

['Tell me something about weather in 2018.', 'In 2018, there were several significant weather events around the world. For example, there were devastating wildfires in California, record-breaking heatwaves in Europe, and powerful hurricanes in the Atlantic. Overall, 2018 was one of the hottest years on record globally, with many regions experiencing extreme weather patterns. Is there anything specific you would like to know about the weather in 2018?']
['Were there any particularly long lasting storms in 2018?', 'Yes, in 2018 there were several long-lasting storms that made headlines. For example, Hurricane Florence lingered over the Carolinas for days, causing catastrophic flooding. Additionally, Typhoon Mangkhut in the western Pacific was one of the strongest storms of the year and caused widespread damage in the Philippines and China. These storms were just a few examples of the extreme weather events that occurred in 2018.']
['What were we just talking about?', 'We were discussing 